In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
from imblearn.under_sampling import NearMiss
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from scipy import stats
from scipy.stats import chi2
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dengue2013/casos-dengue2013.csv
/kaggle/input/dengue2017/dengue2017.csv
/kaggle/input/dengue2015/dengue2015.csv
/kaggle/input/dengue2014/casos-dengue2014.csv
/kaggle/input/dengue2019/dengue2019.csv
/kaggle/input/dengue2016/dengue2016.csv
/kaggle/input/dengue2020/dengue2020.csv
/kaggle/input/dengue2018/dengue2018.csv


In [2]:
#pre-processing data, delimiter ";" to ","
def read_file(ext,file_name):
    reader = csv.reader(open(ext, "rU"), delimiter=';')
    writer = csv.writer(open(file_name, 'w'), delimiter=',')
    writer.writerows(reader)
    data= pd.read_csv("/kaggle/working/"+ file_name)
    data=data[['tp_sexo','febre','mialgia','cefaleia','exantema','vomito',
     'nausea','dor_costas','conjutivite','artrite','artralgia','petequia_n','leucopenia',
     'dor_retro','tp_classificacao_final']]
 
    data=data.fillna('n')
    print('DATA SIZE', len(data))
    data=data.replace(to_replace =['I'],value='n')
    data=data[data["tp_classificacao_final"]!='n']
    return data


In [3]:
def select_antibody_test(data):
    dt=data.copy()
    del dt['tp_result_rtpcr']
    
    teste_ns1=dt[(dt["Tp_result_NS1"]==1) | (dt["Tp_result_NS1"]==2)]

    teste_exame=dt[(dt["tp_result_exame"]==1) | (dt["tp_result_exame"]==2)]
    del teste_exame['Tp_result_NS1']
   
    del teste_ns1['tp_result_exame']
   
    
    teste_rapido=pd.concat([teste_ns1,teste_exame]).drop_duplicates()
    teste_rapido=teste_rapido.fillna('n')
    del teste_rapido['Tp_result_NS1']
    del teste_rapido['tp_result_exame']
    teste_rapido=teste_rapido.drop_duplicates()
    return  teste_rapido


In [4]:
def set_date_exam(teste_rapido):
    
    dt_ns1=teste_rapido[teste_rapido['Tp_result_NS1']!='n']
    dt_ns1=dt_ns1['dt_coleta_NS1'].values
    dt_exame=teste_rapido[teste_rapido['tp_result_exame']!='n']
    dt_exame=dt_exame['dt_coleta_exame'].values
    teste_rapido['dt_coleta']=None
    teste_rapido.iloc[:len(dt_ns1),20:]=dt_ns1
    teste_rapido.iloc[len(dt_ns1):,20:]=dt_exame
    
    #deleting unused columns
    del teste_rapido['dt_coleta_NS1']
    del teste_rapido['dt_coleta_exame']
    return teste_rapido
#teste_rapido= set_date_exam(teste_rapido)

In [5]:
def select_patients_with_symp(data):
    data=data[(data['febre']!='n') & (data['mialgia']!='n') & (data['cefaleia']!='n')&
    (data['exantema']!='n') & (data['vomito']!='n') & (data['nausea']!='n') &
    (data['dor_costas'] !='n') & (data['conjutivite']!='n') & (data['artrite'] !='n') &
    (data['artralgia'] !='n' ) & (data['petequia_n'] !='n' ) & (data['leucopenia']!='n')
    & (data['dor_retro']!='n')]
    return data


In [6]:
def select_pcr_test(data):
    dt=data.copy()
    del dt['Tp_result_NS1']
    

    del dt['tp_result_exame']
    

    teste_pcr=dt[(dt["tp_result_rtpcr"]==1)| (dt["tp_result_rtpcr"]==2)]
    
    del teste_pcr["tp_result_rtpcr"]
    return  teste_pcr


In [7]:
def convert_to_binary(dt):
    dt=dt.replace(to_replace =[1],value =0) 
    dt=dt.replace(to_replace =[2],value =1)
    dt=dt.replace(to_replace =['F'],value=0) 
    dt=dt.replace(to_replace =['M'],value=1)
    dt=dt.replace(to_replace =[10],value=0)
    dt=dt.replace(to_replace =[11],value=0)
    dt=dt.replace(to_replace =[12],value=0)
    
    
    dt=dt.replace(to_replace =[3],value=0)
    dt=dt.replace(to_replace =[4],value=0)
    
    dt=dt.replace(to_replace =[5],value=1)
    dt=dt.replace(to_replace =[8],value=1)
    
    #drop none values
    dt=dt.replace(to_replace =["n"],value =None)
    dt=dt.dropna()
    return dt


In [8]:
def select_classif(data):
    return data[(data['tp_classificacao_final']!=8)]

In [9]:
#2015
data_2015=read_file('/kaggle/input/dengue2015/dengue2015.csv','dengue2015.csv')
data_2015=select_patients_with_symp(data_2015)
data_2015=select_classif(data_2015)
#teste_rapido_2015=select_antibody_test(data_2015)
#teste_pcr_2015=select_pcr_test(data_2015)
#repeted_2015=teste_rapido_2015[teste_rapido_2015.index.isin(teste_pcr_2015.index)]
#teste_rapido_2015=teste_rapido_2015.drop(repeted_2015.index)

#2016
data_2016=read_file('/kaggle/input/dengue2016/dengue2016.csv','dengue2016.csv')
data_2016=select_patients_with_symp(data_2016)
data_2016=select_classif(data_2016)
#teste_rapido_2016=select_antibody_test(data_2016)
#teste_pcr_2016=select_pcr_test(data_2016)
#repeted_2016=teste_rapido_2016[teste_rapido_2016.index.isin(teste_pcr_2016.index)]
#teste_rapido_2016=teste_rapido_2016.drop(repeted_2016.index)

#2017
data_2017=read_file('/kaggle/input/dengue2017/dengue2017.csv','dengue2017.csv')
data_2017=select_patients_with_symp(data_2017)
data_2017=select_classif(data_2017)
#teste_rapido_2017=select_antibody_test(data_2017)
#teste_pcr_2017=select_pcr_test(data_2017)
#repeted_2017=teste_rapido_2017[teste_rapido_2017.index.isin(teste_pcr_2017.index)]
#teste_rapido_2017=teste_rapido_2017.drop(repeted_2017.index)

#2018
data_2018=read_file('/kaggle/input/dengue2018/dengue2018.csv','dengue2018.csv')
data_2018=select_patients_with_symp(data_2018)
data_2018=select_classif(data_2018)
#teste_rapido_2018=select_antibody_test(data_2018)
#teste_pcr_2018=select_pcr_test(data_2018)
#repeted_2020=teste_rapido_2020[teste_rapido_2020.index.isin(teste_pcr_2020.index)]
#teste_rapido_2020=teste_rapido_2020.drop(repeted_2020.index)

#2019
data_2019=read_file('/kaggle/input/dengue2019/dengue2019.csv','dengue2019.csv')
data_2019=select_patients_with_symp(data_2019)
data_2019=select_classif(data_2019)
#teste_rapido_2019=select_antibody_test(data_2019)
#teste_pcr_2019=select_pcr_test(data_2019)
#repeted_2019=teste_rapido_2019[teste_rapido_2019.index.isin(teste_pcr_2019.index)]
#teste_rapido_2019=teste_rapido_2019.drop(repeted_2019.index)

#2020
data_2020=read_file('/kaggle/input/dengue2020/dengue2020.csv','dengue2020.csv')
data_2020=select_patients_with_symp(data_2020)
data_2020=select_classif(data_2020)
#teste_rapido_2020=select_antibody_test(data_2020)
#teste_pcr_2020=select_pcr_test(data_2020)
#repeted_2020=teste_rapido_2020[teste_rapido_2020.index.isin(teste_pcr_2020.index)]
#teste_rapido_2020=teste_rapido_2020.drop(repeted_2020.index)

#concat
teste=pd.concat([data_2019,data_2020]).reset_index()
del teste['index']
#teste_pcr=pd.concat([teste_pcr_2015,teste_pcr_2016,teste_pcr_2017,teste_pcr_2018,teste_pcr_2019,teste_pcr_2020]).reset_index()
#del teste_pcr['index']
teste=convert_to_binary(teste)
#teste_rapido=convert_to_binary(teste_rapido)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'U' mode is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (30,52,96,112) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


DATA SIZE 34533


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (54,112) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


DATA SIZE 18595
DATA SIZE 2442


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'U' mode is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'U' mode is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


DATA SIZE 2662
DATA SIZE 7286
DATA SIZE 1000


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'U' mode is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
 34533+18595+2442+ 2662+7286+1000

66518

In [11]:
teste=teste.rename(columns={"tp_sexo":'Gender',"febre":"Symptom- Fever","mialgia":"Symptom- Myalgia",
                      "cefaleia":"Symptom- Headache","exantema":"Symptom- exanthema",
                      "vomito":"Symptom- Vomit","nausea":"Symptom- Nausea","dor_costas":"Symptom- backache",
                      "conjutivite":"Symptom- conjunctivitis","artrite":"Symptom- arthritis",
                      "artralgia":"Symptom- arthralgia","petequia_n":"Symptom- petechia",
                      "leucopenia":"Symptom- leucopenia","dor_retro":"Symptom- retro_pain"})

In [12]:
len(teste)

7422

In [13]:
teste_fem=teste[teste['Gender']==0]
teste_ma=teste[teste['Gender']==1]

In [14]:
def part_dataset(dt):
    
    #part dataset with only class) marked as negative
    neg=dt[dt["tp_classificacao_final"]==1]
    
    #part dataset with only class marked as positive
    pos=dt[dt["tp_classificacao_final"]==0]
    del neg["tp_classificacao_final"]
    del pos["tp_classificacao_final"]
    
    #part dataset with only unique values fom dataset pos
    pos_uniq=pos.drop_duplicates(subset=['Gender', 'Symptom- Fever', 'Symptom- Myalgia',
                                         'Symptom- Headache','Symptom- exanthema', 'Symptom- Vomit',
                                         'Symptom- Nausea','Symptom- backache', 'Symptom- conjunctivitis',
                                         'Symptom- arthritis','Symptom- arthralgia', 'Symptom- petechia',
                                         'Symptom- leucopenia','Symptom- retro_pain'])
    
    neg_array=neg.iloc[:,:].values
    pos_uniq_array=pos_uniq.iloc[:,:].values
    pos_dupli_array=pos.iloc[:,:].values
    
    return neg,pos,neg_array, pos_uniq_array,pos_dupli_array

neg_fem,pos_fem,neg_array_fem,pos_uniq_array_fem,pos_dupli_array_fem=part_dataset(teste_fem)  
neg_ma,pos_ma,neg_array_ma,pos_uniq_array_ma,pos_dupli_array_ma=part_dataset(teste_ma)

In [15]:
def resolve_class_conflict(d_ori,pos_uniq_array,l_comp):
    lpp=dict()
    dict_count=dict()
    #run all the unique values from positive dataset
    for k in range(len(pos_uniq_array)):
        p=0
        lista_index=[]
        #run all the values to compare if is equal to the unique values from positive dataset
        for i in range(len(l_comp)):

            comparison=pos_uniq_array[k:k+1,:]==l_comp[i:i+1,:]
             #if the values are equal
            if(comparison.all()):
                #count how many equal values exist!
                p+=1
                #add l_comp value index in lista_index!
                lista_index.append(d_ori.iloc[i:i+1,:].index[0])
        #hold the number of same values
        dict_count[k]=p
        #hold all the lists of index
        lpp[k]=lista_index
    return dict_count,lpp
#fem
dict_count_fem,lpp_fem=resolve_class_conflict(pos_fem,pos_uniq_array_fem,pos_dupli_array_fem)
dict_count_neg_fem,lpn_fem=resolve_class_conflict(neg_fem,pos_uniq_array_fem, neg_array_fem)
#mal
dict_count_ma,lpp_ma=resolve_class_conflict(pos_ma,pos_uniq_array_ma,pos_dupli_array_ma)
dict_count_neg_ma,lpn_ma=resolve_class_conflict(neg_ma,pos_uniq_array_ma, neg_array_ma)


In [16]:
def get_conflict_values_keys(dict_count_neg,dict_count):
    dnp=pd.DataFrame(index=dict_count_neg.keys())
    dnp['neg']=dict_count_neg.values()
    dnp['pos']=dict_count.values()
    dnp=dnp[(dnp['neg']>0) & (dnp['pos']>0)]
    #get just the keys which the total rows number of neg values is less than pos values
    neg_smaller=dnp[dnp['neg'] < dnp['pos']]
    #get just the keys which the total rows number of pos values is less than neg values
    pos_smaller=dnp[dnp['neg'] > dnp['pos']]
    #get just the keys which the total rows number of pos values is equal to neg values
    pos_neg_equal=dnp[dnp['neg'] == dnp['pos']]
    return neg_smaller, pos_smaller, pos_neg_equal
#fem
neg_smaller_fem,pos_smaller_fem, pos_neg_equal_fem=get_conflict_values_keys(dict_count_neg_fem,dict_count_fem)
#mal
neg_smaller_ma,pos_smaller_ma, pos_neg_equal_ma=get_conflict_values_keys(dict_count_neg_ma,dict_count_ma)


In [17]:
def get_index(data,l_data,l_index):
    
    for x in data.index:
        for k in l_data[x]:
            l_index.append(k)
    
       
    return l_index
#fem
l_index_fem=[]
l_index_fem=get_index(neg_smaller_fem,lpn_fem,l_index_fem)
l_index_fem=get_index(pos_smaller_fem,lpp_fem,l_index_fem)
l_index_fem=get_index(pos_neg_equal_fem,lpn_fem,l_index_fem)
l_index_fem=get_index(pos_neg_equal_fem,lpp_fem,l_index_fem)
teste_fem=teste_fem.drop(l_index_fem)

#mal
l_index_ma=[]
l_index_ma=get_index(neg_smaller_ma,lpn_ma,l_index_ma)
l_index_ma=get_index(pos_smaller_ma,lpp_ma,l_index_ma)
l_index_ma=get_index(pos_neg_equal_ma,lpn_ma,l_index_ma)
l_index_ma=get_index(pos_neg_equal_ma,lpp_ma,l_index_ma)
teste_ma=teste_ma.drop(l_index_ma)

In [18]:
def match_classes(dt_covid):
    y=dt_covid["tp_classificacao_final"]

    x=dt_covid.iloc[:,:14]
    nr = NearMiss()
    x,y=nr.fit_sample(x,y)
    return x,y
teste_fem_unb = teste_fem.copy()
teste_ma_unb = teste_ma.copy()
x_fem,y_fem=match_classes(teste_fem)
x_fem['Class']=y_fem
teste_fem=x_fem.copy()
#mal
x_ma,y_ma=match_classes(teste_ma)
x_ma['Class']=y_ma
teste_ma=x_ma.copy()


In [19]:
teste_ma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   1340 non-null   object 
 1   Symptom- Fever           1340 non-null   float64
 2   Symptom- Myalgia         1340 non-null   float64
 3   Symptom- Headache        1340 non-null   float64
 4   Symptom- exanthema       1340 non-null   float64
 5   Symptom- Vomit           1340 non-null   float64
 6   Symptom- Nausea          1340 non-null   float64
 7   Symptom- backache        1340 non-null   float64
 8   Symptom- conjunctivitis  1340 non-null   float64
 9   Symptom- arthritis       1340 non-null   float64
 10  Symptom- arthralgia      1340 non-null   float64
 11  Symptom- petechia        1340 non-null   float64
 12  Symptom- leucopenia      1340 non-null   float64
 13  Symptom- retro_pain      1340 non-null   float64
 14  Class                   

In [20]:
teste_fem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384 entries, 0 to 1383
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   1384 non-null   object 
 1   Symptom- Fever           1384 non-null   float64
 2   Symptom- Myalgia         1384 non-null   float64
 3   Symptom- Headache        1384 non-null   float64
 4   Symptom- exanthema       1384 non-null   float64
 5   Symptom- Vomit           1384 non-null   float64
 6   Symptom- Nausea          1384 non-null   float64
 7   Symptom- backache        1384 non-null   float64
 8   Symptom- conjunctivitis  1384 non-null   float64
 9   Symptom- arthritis       1384 non-null   float64
 10  Symptom- arthralgia      1384 non-null   float64
 11  Symptom- petechia        1384 non-null   float64
 12  Symptom- leucopenia      1384 non-null   float64
 13  Symptom- retro_pain      1384 non-null   float64
 14  Class                   

In [21]:
both_dengue_data=pd.concat([teste_fem,teste_ma]).reset_index()
del both_dengue_data['index']
both_dengue_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2724 entries, 0 to 2723
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   2724 non-null   object 
 1   Symptom- Fever           2724 non-null   float64
 2   Symptom- Myalgia         2724 non-null   float64
 3   Symptom- Headache        2724 non-null   float64
 4   Symptom- exanthema       2724 non-null   float64
 5   Symptom- Vomit           2724 non-null   float64
 6   Symptom- Nausea          2724 non-null   float64
 7   Symptom- backache        2724 non-null   float64
 8   Symptom- conjunctivitis  2724 non-null   float64
 9   Symptom- arthritis       2724 non-null   float64
 10  Symptom- arthralgia      2724 non-null   float64
 11  Symptom- petechia        2724 non-null   float64
 12  Symptom- leucopenia      2724 non-null   float64
 13  Symptom- retro_pain      2724 non-null   float64
 14  Class                   

In [22]:
teste_ma_unb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2360 entries, 4 to 7421
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   2360 non-null   object 
 1   Symptom- Fever           2360 non-null   float64
 2   Symptom- Myalgia         2360 non-null   float64
 3   Symptom- Headache        2360 non-null   float64
 4   Symptom- exanthema       2360 non-null   float64
 5   Symptom- Vomit           2360 non-null   float64
 6   Symptom- Nausea          2360 non-null   float64
 7   Symptom- backache        2360 non-null   float64
 8   Symptom- conjunctivitis  2360 non-null   float64
 9   Symptom- arthritis       2360 non-null   float64
 10  Symptom- arthralgia      2360 non-null   float64
 11  Symptom- petechia        2360 non-null   float64
 12  Symptom- leucopenia      2360 non-null   float64
 13  Symptom- retro_pain      2360 non-null   float64
 14  tp_classificacao_final  

In [23]:
teste_fem_unb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2678 entries, 0 to 7420
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   2678 non-null   object 
 1   Symptom- Fever           2678 non-null   float64
 2   Symptom- Myalgia         2678 non-null   float64
 3   Symptom- Headache        2678 non-null   float64
 4   Symptom- exanthema       2678 non-null   float64
 5   Symptom- Vomit           2678 non-null   float64
 6   Symptom- Nausea          2678 non-null   float64
 7   Symptom- backache        2678 non-null   float64
 8   Symptom- conjunctivitis  2678 non-null   float64
 9   Symptom- arthritis       2678 non-null   float64
 10  Symptom- arthralgia      2678 non-null   float64
 11  Symptom- petechia        2678 non-null   float64
 12  Symptom- leucopenia      2678 non-null   float64
 13  Symptom- retro_pain      2678 non-null   float64
 14  tp_classificacao_final  

In [24]:
both_dengue_data_unb =pd.concat([teste_fem_unb,teste_ma_unb]).reset_index()
del both_dengue_data_unb['index']
both_dengue_data_unb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5038 entries, 0 to 5037
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   5038 non-null   object 
 1   Symptom- Fever           5038 non-null   float64
 2   Symptom- Myalgia         5038 non-null   float64
 3   Symptom- Headache        5038 non-null   float64
 4   Symptom- exanthema       5038 non-null   float64
 5   Symptom- Vomit           5038 non-null   float64
 6   Symptom- Nausea          5038 non-null   float64
 7   Symptom- backache        5038 non-null   float64
 8   Symptom- conjunctivitis  5038 non-null   float64
 9   Symptom- arthritis       5038 non-null   float64
 10  Symptom- arthralgia      5038 non-null   float64
 11  Symptom- petechia        5038 non-null   float64
 12  Symptom- leucopenia      5038 non-null   float64
 13  Symptom- retro_pain      5038 non-null   float64
 14  tp_classificacao_final  

In [25]:
both_dengue_data.to_csv('both_dengue_data.csv',index=False)
teste_fem.to_csv('fem_dengue_data.csv',index=False)
teste_ma.to_csv('ma_dengue_data.csv',index=False)

both_dengue_data_unb.to_csv('both_dengue_data_unb.csv',index=False)
teste_fem_unb.to_csv('fem_dengue_data_unb.csv',index=False)
teste_ma_unb.to_csv('ma_dengue_data_unb.csv',index=False)